# Benchmarking a method

When developing a new method, you might want to see how other methods perform
on the dataset you are evaluating. Photoholmes includes the implementation for several
methods from the literature, covering a diverse array of approaches to forgery detection
to compare to.

It also includes a Benchmark object to easily evaluate the performance of a method over
a dataset. This ensures a fair and reproducible comparison between methods.
This notebook is a short tutorial on how to use Benchmark object, using as an example 
the included method DQ and a custom method we will define on the Columbia dataset.

1. [ Running an included method ](#running-an-included-method)
    - [Method selection](#method-selection)
    - [Loading the dataset](#loading-a-dataset-for-evaluation)
    - [Selecting the metrics](#selecting-the-metrics)
    - [Benchmark evaluation](#benchmark-evaluation)
    
2. [ Running a custom method ](#running-a-custom-method)


## Running an included method


### Method selection

To run an included method, we first import and instantiate the method. You can do this by using the **method factory**.


In [ ]:
from photoholmes.methods import MethodFactory, MethodRegistry


dq, dq_preprocessing = MethodFactory.load(MethodRegistry.DQ)

Or by **importing** the method **directly**. In this case, it's important to also import the corresponding **preprocessing** pipeline, as it is a required argument for the Dataset. 


In [ ]:
from photoholmes.methods.dq import DQ, dq_preprocessing

dq = DQ()

For more information on about the methods and how to use them, you can check the [methods documentation](
    ../SRC/photoholmes/methods/README.md)

### Loading a Dataset for evaluation

To continue, we must first download the [Columbia Uncompressed Image Splicing Detection](https://www.ee.columbia.edu/ln/dvmm/downloads/authsplcuncmp/) dataset. Keep in mind this dataset is under a [ research-only use License ](https://www.ee.columbia.edu/ln/dvmm/downloads/authsplcuncmp/dlform.html). You can download the dataset [ here ](https://www.dropbox.com/sh/786qv3yhvc7s9ki/AACbEEzGPrD3_y38bpWHzgdqa?e=1&dl=0).

Once downloaded, unzip the files and update the following variable with the path to the dataset folder.


In [ ]:
columbia_dataset_path: str = (
    "../data/Benchmark/Columbia Uncompressed Image Splicing Detection"  # UPDATE WITH THE PATH ON YOUR COMPUTER
)

As well as with the methods, we can load a dataset by **direct import**:


In [ ]:
from photoholmes.datasets.columbia import ColumbiaDataset

dataset = ColumbiaDataset(
    dataset_path=columbia_dataset_path,
    load=["image", "dct_coefficients"],
    preprocessing_pipeline=dq_preprocessing,
)
print("Total images: ", len(dataset))

Or using the **factory**


In [ ]:
from photoholmes.datasets import DatasetFactory, DatasetRegistry


dataset = DatasetFactory.load(
    DatasetRegistry.COLUMBIA,
    dataset_path=columbia_dataset_path,
    load=["image", "dct_coefficients"],
    preprocessing_pipeline=dq_preprocessing,
)
print("Total images: ", len(dataset))

For more information on about the datasets and how to use them, you can check the [datasets documentation](
    ../src/photoholmes/datasets/README.md)

### Selecting the metrics

Lastly, we need to select the metrics to evaluate. We will load the Auroc, IoU and F1 using the MetricFactory which allows us to easily select the metrics we want to use.

In [ ]:
from photoholmes.metrics.factory import MetricFactory

metrics = MetricFactory.load(["auroc", "f1", "iou"])
print(metrics)

For more information on about the metrics and how to use them, you can check the [metrics documentation](
    ../src/photoholmes/metrics/README.md)

### Benchmark evaluation

Now, we are ready to run the Benchmark. First, we create a Benchmark Object. The constructor allows to tune the following parameters:

- **save_method_outputs:** Whether to save the method outputs.
- **save_extra_outputs:** Whether to save extra outputs.
- **save_metrics:** Whether to save metrics.
- **output_folder:** Path to the output folder.
- **device:** torch Device for computation.
- **use_existing_output:** Whether to use existing saved outputs.
- **verbose:** Verbosity level.


In [ ]:
from photoholmes.benchmark import Benchmark

benchmark = Benchmark(
    save_method_outputs=True,
    save_extra_outputs=False,
    save_metrics=True,
    output_folder="example_output",
    device="cpu",
    use_existing_output=False,
    verbose=1,
)

We are ready to go! The following cell will run the evaluation. It should take around two minutes to continue.


In [ ]:
dq_results = benchmark.run(
    method=dq,
    dataset=dataset,
    metrics=metrics,
)
print(dq_results)

Notice that a folder example_output has been created. There, the benchmark will create the following folder structure:

```terminal
example_output/
└── dq
    └── columbiadataset
        ├── metrics
        │   └── <timestamp>_tampered_and_pristine
        │       ├── heatmap_report.json
        │       └── heatmap_state.pt
        └── outputs
            ├── canong3_02_sub_01
            │   └── output.npz
            ├── canong3_02_sub_02
            ...
```

Inside the _\<method\>_\/_\<dataset\>_ folder, in this case _dq/colmubiadataset_, you will find two folders: _output/_ and _metrics_. Inside the ouputs
folder you will find the saved model outputs, so they can be reused and save compute time. There are three types of output:

1. _output.npz_: saves the benchmark outputs (heatmap, mask and/or detection)
2. _output_extra.npz_: saves any extra output arrays that were included in the benchmark output. This will be included only if the benchmark has the
   _save_extra_output=True_.
3. _output_extra_json.json_: save any extra output that isn't an array.

On the metrics folder, you will find the benchmarking results. Every time you run a benchmark, a folder with the name _\<timestamp\>_ _ _\<dataset_type\>_ is created (the latter being _tampered only_ or _tampered and pristine_). Inside the folder, you will find metric files for each type of output your method outputs.
Photoholmes divides method outputs into three types:

1. **heatmap:** a probability map.
2. **mask:** a binary mask.
3. **detection:** a score for detection.

Your method can predict only one output per type. Inside the metrics folder you will find a report for each type your method outputed, in this case
only _heatmap_report.json_.


Congrats! You have benchmarked DQ on the columbia dataset.


## Running a custom method


First, let's implemented a basic method to use our benchmark on. We won't do anything fancy, simply predict a random array in the same shape
as the image.

To do so, we should create a child class from _BaseMethod_ and overide the _\_\_init\_\__, _predict_ and _benchmark_ to the desired behaviour. In our notation, predict is the class method that performs the prediction of the heatmap (mask, detection or others), and benchmark method returns these outputs in the form of a dictionary with standardized keys. For more details, see the documentation. 


In [ ]:
import random
from typing import Tuple
from photoholmes.methods.base import BaseMethod, BenchmarkOutput
import numpy as np
from numpy.typing import NDArray
import torch
from photoholmes.preprocessing.image import ToNumpy

from photoholmes.preprocessing.pipeline import PreProcessingPipeline


class RandomMethod(BaseMethod):

    def __init__(self, threshold=0.5):
        super().__init__()
        self.threshold = threshold

    def predict(self, image: NDArray) -> Tuple[NDArray, NDArray, float]:
        heatmap = np.random.random(size=image.shape[:2])
        detection = random.random()
        mask = heatmap>self.threshold
        return heatmap, mask, detection

    def benchmark(self, image: NDArray) -> BenchmarkOutput:
        heatmap, mask, detection = self.predict(image)
        return {
            "heatmap": torch.from_numpy(heatmap),
            "mask": torch.from_numpy(mask),
            "detection": torch.tensor([detection]),
        }


method = RandomMethod()
random_method_preprocessing = PreProcessingPipeline(
    [ToNumpy()],
    inputs=["image"],
    outputs_keys=["image"]
)

Let's test it out in an image of Paul McCartney drinking "fernet". The image is downloaded from [Taringa](https://media.taringa.net/knn/fit:550/Z3M6Ly9rbjMvdGFyaW5nYS9ELzIvNy9GLzQvRi9veWVjb21vdmFhLzQzNy5qcGc)


In [ ]:
! mkdir -p data
! curl https://media.taringa.net/knn/fit:550/Z3M6Ly9rbjMvdGFyaW5nYS9ELzIvNy9GLzQvRi9veWVjb21vdmFhLzQzNy5qcGc -o data/paul.webp

In [ ]:
from photoholmes.utils.image import read_image, plot

img = read_image("data/paul.webp")
plot(img)

In [ ]:
from photoholmes.utils.image import plot_multiple

input_preprocessed = random_method_preprocessing(image=img)
heatmap, mask, score = method.predict(**input_preprocessed)
print("Detection score:", score)
plot_multiple([heatmap, mask], titles=["Heatmap", "Mask"], title="Random Method")

Our method clearly isn't good, but it serves as a good enough example to test the benchmark functionality.

Let's load our datasets and metrics again and run the benchmark.

In [ ]:
from photoholmes.datasets.factory import DatasetFactory, DatasetRegistry
from photoholmes.metrics.factory import MetricFactory, MetricRegistry


columbia_dataset_path: str = (
    "data/Columbia Uncompressed Image Splicing Detection"  # UPDATE WITH THE PATH ON YOUR COMPUTER
)

dataset = DatasetFactory.load(
    dataset_name=DatasetRegistry.COLUMBIA,
    dataset_path=columbia_dataset_path,
    load=["image", "dct_coefficients"],
    preprocessing_pipeline=random_method_preprocessing,
)
print("Total images: ", len(dataset))

metrics = MetricFactory.load([MetricRegistry.AUROC, MetricRegistry.F1, MetricRegistry.IoU])
print(metrics)

In [ ]:
from photoholmes.benchmark import Benchmark

benchmark = Benchmark(output_folder="example_output")

In [ ]:
random_method_results = benchmark.run(method=method, dataset=dataset, metrics=metrics)
print(random_method_results)

If you check the output folder, you will notice a folder _randommethod/columbiadataset_ with the evaluation results.